In [25]:
import pandas as pd
import numpy as np
import random

from rich.console import Console

from bokeh.plotting import figure, show, curdoc
from bokeh.models import Select, ColumnDataSource
from bokeh.io import output_notebook, curdoc, output_file
from bokeh.layouts import row, column, gridplot
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv(r'data/EERecords.csv')

In [5]:
loc_df = pd.read_csv(r'data/EEtraplocs.csv')

In [6]:
cam_df = pd.read_csv(r'data/EECam_act.csv')

In [8]:
rc = Console()

In [9]:
df.head()

,Ind,Trap_ID,Occasion,Primary Period
0,1,4,44,1
1,1,4,52,1
2,1,5,67,1
3,1,5,72,1
4,1,6,73,1


##### Number of Tigers spotted

In [10]:
df.Ind.nunique()

75

##### Number of cameras

In [11]:
tot_cams = loc_df.Trap_ID.nunique()
act_cams = df.Trap_ID.nunique()

rc.print("Total cameras : ", tot_cams, style="yellow")
rc.print("Active cameras : ", act_cams, style="green")
rc.print("Inactive cameras : ", tot_cams-act_cams, style="red")

Total cameras :  195

Active cameras :  111

Inactive cameras :  84

In [12]:
df['Primary Period'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

### Location

In [13]:
df1 = df.merge(loc_df, on='Trap_ID', how='inner')

In [14]:
df2 = df1.merge(cam_df,)

In [15]:
df1

,Ind,Trap_ID,Occasion,Primary Period,CenteredX,CenteredY
0,1,4,44,1,-3259.010435,1548.942538
1,1,4,52,1,-3259.010435,1548.942538
2,3,4,61,1,-3259.010435,1548.942538
3,5,4,39,1,-3259.010435,1548.942538
4,1,4,39,2,-3259.010435,1548.942538
...,...,...,...,...,...,...
338,32,194,3,10,-3045.725635,2685.386538
339,55,152,23,10,2049.036065,5824.901538
340,67,191,43,10,4598.588465,-6076.825462
341,68,191,43,10,4598.588465,-6076.825462


In [16]:
cams = [str(i) for i in df1['Trap_ID'].unique()]

In [17]:
type(cams[0])

str

In [20]:


# List of predefined Matplotlib color maps (palettes)
color_maps = plt.colormaps()

def fill_palettes(num):
    """
    Get a list of random color palettes for charts.

    Parameters:
    - num (int): The number of random color palettes to generate.

    Returns:
    - List of random color palettes.
    """
    if num > len(color_maps):
        raise ValueError("Number of requested palettes exceeds the available palettes.")

    # Shuffle the list of color palettes randomly
    random.shuffle(color_maps)

    # Select the first `num_palettes` palettes from the shuffled list
    selected = color_maps[:num]

    return selected

# Example usage:
palettes = fill_palettes(75)
print(palettes)


['afmhot', 'PuOr_r', 'Set2', 'OrRd_r', 'twilight_shifted_r', 'Wistia_r', 'pink_r', 'copper_r', 'terrain_r', 'ocean', 'RdBu', 'gist_ncar_r', 'magma_r', 'hot', 'copper', 'cubehelix_r', 'tab20b', 'PuBu_r', 'CMRmap_r', 'BrBG', 'autumn', 'gray_r', 'Oranges', 'Set2_r', 'Dark2_r', 'gist_heat', 'turbo', 'PuOr', 'gnuplot2_r', 'turbo_r', 'YlOrRd', 'Dark2', 'gist_ncar', 'inferno', 'Set3', 'twilight', 'binary_r', 'Reds_r', 'Pastel1', 'RdGy_r', 'spring_r', 'gnuplot_r', 'tab20c_r', 'Blues_r', 'viridis_r', 'bone', 'plasma_r', 'YlOrBr_r', 'gist_rainbow', 'Set1_r', 'Accent_r', 'terrain', 'hot_r', 'YlGn_r', 'afmhot_r', 'hsv_r', 'cool_r', 'bwr_r', 'Greens_r', 'YlOrBr', 'Purples_r', 'gist_gray_r', 'cividis_r', 'Greys_r', 'brg', 'cividis', 'YlGnBu', 'Purples', 'GnBu_r', 'seismic', 'BuPu_r', 'gist_yarg_r', 'YlOrRd_r', 'Pastel2_r', 'Paired']


['1',
 '3',
 '5',
 '12',
 '20',
 '9',
 '2',
 '10',
 '4',
 '7',
 '13',
 '14',
 '21',
 '16',
 '8',
 '11',
 '23',
 '25',
 '19',
 '32',
 '58',
 '22',
 '15',
 '34',
 '51',
 '54',
 '61',
 '18',
 '17',
 '24',
 '26',
 '62',
 '74',
 '31',
 '33',
 '43',
 '48',
 '35',
 '30',
 '39',
 '40',
 '55',
 '76',
 '47',
 '38',
 '64',
 '75',
 '37',
 '60',
 '27',
 '66',
 '28',
 '29',
 '72',
 '46',
 '50',
 '71',
 '42',
 '53',
 '41',
 '56',
 '57',
 '36',
 '44',
 '63',
 '45',
 '49',
 '67',
 '68',
 '70',
 '73',
 '52',
 '65',
 '59',
 '69']

In [38]:
tiger_sel = Select(
    title="Select Tiger",
    value='1',
    options=[str(i) for i in df1['Ind'].unique()]
)

map_line = ColumnDataSource(df1)

# Create a new plot with a title and axis labels
p = figure(
    title="Area chart",
    x_axis_label='X',
    y_axis_label='Y'
)

# Add a scatter glyph with X and Y data
p.scatter(
    source=loc_df,
    x='CenteredX',
    y='CenteredY',
    size=10,
    alpha=0.5
)
p.line(
    source=map_line,
    x='CenteredX',
    y='CenteredY',
    line_width=2,
    legend_label='Ind'
)

# - Function
def tiger_line(attr, old, new):
    """Select continent name"""
    id = tiger_sel.value
    map_line.data = df1.query('continent==id')

tiger_sel.on_change('value', tiger_line)
tiger_chart = gridplot([[tiger_sel], [p]])

# Output the plot to a Jupyter Notebook
output_notebook()
show(tiger_chart)

Loading BokehJS ...

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

